## Import

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.init import xavier_normal_, xavier_uniform_
from torch.optim.lr_scheduler import ReduceLROnPlateau

import argparse
import numpy as np
import time
import sys
import pandas as pd
from os.path import abspath
from scipy.sparse import coo_matrix
import scipy.sparse as sp
import os
import numpy as np 
import time 
from itertools import product

from util.sampler import next_batch_pairwise_kg, next_batch_pairwise
from util.conf import OptionConf
from base.torch_interface import TorchGraphInterface


from data.loader import FileIO
from util.conf import ModelConf
from base.recommender import Recommender
from util.algorithm import find_k_largest
from time import strftime, localtime
from data.loader import FileIO
from util.evaluation import ranking_evaluation

from base.graph_recommender import GraphRecommender
from model.layers.graph_transformer import *
from util.loss_torch import bpr_loss, l2_reg_loss, EmbLoss, contrastLoss, l2_reg_loss
from util.init import *
from data.ui_graph import Interaction

/home/jun/anaconda3/envs/hungvv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

## Base Recommender

In [13]:
class GraphRecommender(Recommender):
    def __init__(self, conf, training_set, test_set, knowledge_set, **kwargs):
        super(GraphRecommender, self).__init__(conf, training_set, test_set, knowledge_set, **kwargs)
        self.data = GraphInteraction(conf, training_set, test_set, knowledge_set, self.knowledge)
        self.bestPerformance = []
        top = self.ranking['-topN'].split(',')
        self.topN = [int(num) for num in top]
        self.max_N = max(self.topN)
        
        self.output = f"./results/LPHGTransformer/{self.dataset}/@{self.model_name}-inp_emb:{kwargs['input_dim']}-hyper_emb:{kwargs['hyper_dim']}-bs:{self.batch_size}-lr:{kwargs['lr']}-lrd:{kwargs['lr_decay']}-reg:{kwargs['reg']}-leaky:{kwargs['p']}-dropout:{kwargs['drop_rate']}-n_layers:{kwargs['n_layers']}-n_heads:{kwargs['n_heads']}-n_self_att:{kwargs['n_self_att']}/"
        if not os.path.exists(self.output):
            os.makedirs(self.output)

    def print_model_info(self):
        super(GraphRecommender, self).print_model_info()
        # # print dataset statistics
        print('Training Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.training_size()))
        print('Test Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.test_size()))
        print('=' * 80)

    def build(self):
        pass

    def train(self):
        pass

    def predict(self, u):
        pass

    def test(self, user_emb, item_emb):
        def process_bar(num, total):
            rate = float(num) / total
            ratenum = int(50 * rate)
            r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
            sys.stdout.write(r)
            sys.stdout.flush()

        # predict
        rec_list = {}
        user_count = len(self.data.test_set)
        for i, user in enumerate(self.data.test_set):
            # s_find_candidates = time.time()
            
            # candidates = predict(user)
            user_id  = self.data.get_user_id(user)
            score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
            candidates = score.cpu().numpy()
            
            # e_find_candidates = time.time()
            # print("Calculate candidates time: %f s" % (e_find_candidates - s_find_candidates))
            # predictedItems = denormalize(predictedItems, self.data.rScale[-1], self.data.rScale[0])
            rated_list, li = self.data.user_rated(user)
            for item in rated_list:
                candidates[self.data.item[item]] = -10e8
            
            # s_find_k_largest = time.time()
            ids, scores = find_k_largest(self.max_N, candidates)
            # e_find_k_largest = time.time()
            # print("Find k largest candidates: %f s" % (e_find_k_largest - s_find_k_largest))
            item_names = [self.data.id2item[iid] for iid in ids]
            rec_list[user] = list(zip(item_names, scores))
            if i % 1000 == 0:
                process_bar(i, user_count)
        process_bar(user_count, user_count)
        print('')
        return rec_list

    def evaluate(self, rec_list):
        self.recOutput.append('userId: recommendations in (itemId, ranking score) pairs, * means the item is hit.\n')
        for user in self.data.test_set:
            line = str(user) + ':'
            for item in rec_list[user]:
                line += ' (' + str(item[0]) + ',' + str(item[1]) + ')'
                if item[0] in self.data.test_set[user]:
                    line += '*'
            line += '\n'
            self.recOutput.append(line)
        current_time = strftime("%Y-%m-%d %H-%M-%S", localtime(time.time()))
        # output prediction result
        out_dir = self.output
        # out_dir = self.output['-dir']
        file_name = self.config['model.name'] + '@' + current_time + '-top-' + str(self.max_N) + 'items' + '.txt'
        FileIO.write_file(out_dir, file_name, self.recOutput)
        print('The result has been output to ', abspath(out_dir), '.')
        file_name = self.config['model.name'] + '@' + current_time + '-performance' + '.txt'
        self.result = ranking_evaluation(self.data.test_set, rec_list, self.topN)
        self.model_log.add('###Evaluation Results###')
        self.model_log.add(self.result)
        FileIO.write_file(out_dir, file_name, self.result)
        print('The result of %s:\n%s' % (self.model_name, ''.join(self.result)))

    def fast_evaluation(self, model, epoch, user_emb, item_emb, kwargs=None):
        print('Evaluating the model...')
        s_test = time.time()
        rec_list = self.test(user_emb, item_emb)
        e_test = time.time() 
        print("Test time: %f s" % (e_test - s_test))
        
        s_measure = time.time()
        all_measures = ranking_evaluation(self.data.test_set, rec_list, self.topN)
        len_measures = len(self.topN)
        lst_res = []
    
        data_ep = {}
        data_ep['epoch'] = epoch
        for i in range(0, len(all_measures), 5):
            mes = all_measures[i:i+5]
            topk = int(mes[0].split(' ')[1][:-1])
            hitk = float(mes[1].split(':')[1][:-1])
            preck = float(mes[2].split(':')[1][:-1])
            reck = float(mes[3].split(':')[1][:-1])
            ndcgk = float(mes[4].split(':')[1][:-1])
            data_ep[f'hit@{topk}'] = hitk
            data_ep[f'precision@{topk}'] = preck
            data_ep[f'recall@{topk}'] = reck
            data_ep[f'ndcg@{topk}'] = ndcgk

        measure = all_measures[(len_measures - 1) * 5: (len_measures - 1) * 5 + 5]
        e_measure = time.time()
        print("Measure time: %f s" % (e_measure - s_measure))
        if len(self.bestPerformance) > 0:
            count = 0
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            for k in self.bestPerformance[1]:
                if self.bestPerformance[1][k] > performance[k]:
                    count += 1
                else:
                    count -= 1
            if count < 0:
                self.bestPerformance[1] = performance
                self.bestPerformance[0] = epoch + 1
                self.save(model)
        else:
            self.bestPerformance.append(epoch + 1)
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            self.bestPerformance.append(performance)
            self.save(model)

        print('-' * 120)
        print('Real-Time Ranking Performance ' + ' (Top-' + str(self.max_N) + ' Item Recommendation)')
        measure = [m.strip() for m in measure[1:]]
        print('*Current Performance*')
        print('Epoch:', str(epoch + 1) + ',', '  |  '.join(measure))
        bp = ''
        # for k in self.bestPerformance[1]:
        #     bp+=k+':'+str(self.bestPerformance[1][k])+' | '
        bp += 'Hit Ratio' + ':' + str(self.bestPerformance[1]['Hit Ratio']) + '  |  '
        bp += 'Precision' + ':' + str(self.bestPerformance[1]['Precision']) + '  |  '
        bp += 'Recall' + ':' + str(self.bestPerformance[1]['Recall']) + '  |  '
        # bp += 'F1' + ':' + str(self.bestPerformance[1]['F1']) + ' | '
        bp += 'NDCG' + ':' + str(self.bestPerformance[1]['NDCG'])
        print('*Best Performance* ')
        print('Epoch:', str(self.bestPerformance[0]) + ',', bp)
        print('-' * 120)
        return measure, data_ep
    
    def save(self, model):
        with torch.no_grad():
            self.best_user_emb, self.best_item_emb, _ = model()
        self.save_model(model)
    
    def save_model(self, model):
        # save model 
        current_time = strftime("%Y-%m-%d", localtime(time.time()))
        out_dir = self.output
        file_name =  self.config['model.name'] + '@' + current_time + '-weight' + '.pth'
        weight_file = out_dir + '/' + file_name 
        torch.save(model.state_dict(), weight_file)

    def save_loss(self, train_losses, rec_losses, reg_losses):
        df_train_loss = pd.DataFrame(train_losses, columns = ['ep', 'loss'])
        df_rec_loss = pd.DataFrame(rec_losses, columns = ['ep', 'loss'])
        df_reg_loss = pd.DataFrame(reg_losses, columns = ['ep', 'loss'])
        df_train_loss.to_csv(self.output + '/train_loss.csv')
        df_rec_loss.to_csv(self.output + '/rec_loss.csv')
        df_reg_loss.to_csv(self.output + '/reg_loss.csv')

    def save_perfomance_training(self, log_train):
        df_train_log = pd.DataFrame(log_train)
        df_train_log.to_csv(self.output + '/train_performance.csv')

## Layers

In [5]:
class HGNNConv(nn.Module):
    def __init__(self, leaky, input_dim, hyper_dim, bias=False):
        super(HGNNConv, self).__init__()
        self.hyper_dim = hyper_dim
        self.act = nn.LeakyReLU(negative_slope=leaky)
        self.fc1 = nn.Linear(input_dim, hyper_dim ,bias=False) 
        self.fc2 = nn.Linear(hyper_dim, hyper_dim ,bias=False)  
        self.fc3 = nn.Linear(hyper_dim, hyper_dim ,bias=False)  
        
        self.ln1 = torch.nn.LayerNorm(hyper_dim)
        self.ln2 = torch.nn.LayerNorm(hyper_dim)
        self.ln3 = torch.nn.LayerNorm(hyper_dim)
        self.ln4 = torch.nn.LayerNorm(hyper_dim)
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(hyper_dim))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
        
    def reset_parameters(self):
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, adj, embeds):
        lat1 = self.act(adj.T @ embeds)
        lat1 = self.ln1(lat1)

        lat2 = self.act(self.fc1(lat1)) +  lat1
        lat2 = self.ln2(lat2)
        
        lat3 = self.act(self.fc2(lat2)) + lat2
        lat3 = self.ln3(lat3)
        
        lat4 = self.act(self.fc3(lat3)) + lat3 
        output = adj @ lat4
        if self.bias is not None:
            output += self.bias 
        output = self.ln4(output)
        ret = self.act(output)
        return ret

## GraphInteraction

In [6]:
class GraphInteraction(Interaction):
    '''
        Input: Interaction -> Output: Graph 
    '''
    def __init__(self, conf, training, test, knowledge_set, knowledge=False):
        Interaction.__init__(self, conf, training, test, knowledge_set, knowledge)
        # import pdb; pdb.set_trace()
        self.lap_pos_enc_ui= self.laplacian_positional_encoding(self.ui_adj, self.conf['pos_enc_dim'])
        self.lap_pos_enc_kg = self.laplacian_positional_encoding(self.kg_ui_adj, self.conf['pos_enc_dim'])
        
        self.num_ntypes_ui= torch.tensor(2, dtype=torch.int32)
        self.num_ntypes_kg= torch.tensor(2, dtype=torch.int32)
        
        num_ui_relations = len(set(np.array(self.training_data)[:,2]))
        num_kg_relations = len(set(np.array(self.training_knowledge_data)[:,2]))
        self.num_etypes_ui= torch.tensor(num_ui_relations, dtype=torch.int32)
        self.num_etypes_kg= torch.tensor(num_kg_relations, dtype=torch.int32)
    
    def laplacian_positional_encoding(self, adj_mat, pos_enc_dim):        
        """
            Graph positional encoding v/ Laplacian eigenvectors
        """
        # Laplacian = I - D^(-1/2)AD^(-1/2) 
        adj_shape = adj_mat.get_shape()
        n_nodes = adj_shape[0]+adj_shape[1]
        (user_np_keep, item_np_keep) = adj_mat.nonzero()
        ratings_keep = adj_mat.data
        tmp_adj = sp.csr_matrix((ratings_keep, (user_np_keep, item_np_keep + adj_shape[0])),shape=(n_nodes, n_nodes),dtype=np.float32)
        tmp_adj = tmp_adj + tmp_adj.T

        rowsum = np.array(tmp_adj.sum(1))
        d_inv = np.power(rowsum, -0.5).flatten()
        d_inv[np.isinf(d_inv)] = 0.
        d_mat_inv = sp.diags(d_inv)
        norm_adj_mat = d_mat_inv.dot(tmp_adj)
        L = sp.eye(n_nodes) - norm_adj_mat * tmp_adj # Lapacian matrix 
        _, EigVec = sp.linalg.eigs(L, k=int(pos_enc_dim))
        lap_pos_encoding = torch.from_numpy(EigVec).float() 
        return lap_pos_encoding
    
    def create_graph_with_pos_enc(self, source_name, target_name, source_embedding, target_embedding, type='interaction', pos_enc='laplacian'):
        if type == 'interaction':
            data = np.array(self.training_data)
            
            users = [self.user[u] for u in data[:,0]]
            items = [self.item[i] for i in data[:,1]]
            
            source_ids = torch.tensor(users, dtype=torch.int32).to(source_embedding.device)
            target_ids = torch.tensor(items, dtype=torch.int32).to(source_embedding.device)
            relation_ids  = torch.tensor(data[:,2], dtype=torch.int32).to(source_embedding.device)
            graph = dgl.heterograph({(source_name, 'interacted', target_name): (source_ids, target_ids)}).to(source_embedding.device)
            graph.nodes[source_name].data['embedding'] =  source_embedding
            graph.nodes[target_name].data['embedding'] =  target_embedding 
            graph.edges['interacted'].data['type'] = relation_ids 
            # if pos_enc == 'laplacian':
            #     graph.ndata['lap_pos_enc'] = self.laplacian_positional_encoding(self.ui_adj, self.conf['pos_enc_dim'])

        elif type == 'knowledge':
            data = np.array(self.training_knowledge_data)
            items = [self.item[i] for i in data[:,0]]
            entities = [ self.entity[i] for i in data[:,1]]

            source_ids =  torch.tensor(items, dtype=torch.int32).to(source_embedding.device)
            target_ids = torch.tensor(entities, dtype=torch.int32).to(source_embedding.device)
            relation_ids  = torch.tensor(data[:,2], dtype=torch.int32).to(source_embedding.device)
            
            graph = dgl.heterograph({(source_name, 'interacted', target_name): (source_ids, target_ids)})
            graph.nodes[source_name].data['embedding'] =  source_embedding
            graph.nodes[target_name].data['embedding'] =  target_embedding 
            graph.edges['interacted'].data['type'] = relation_ids 
            # if pos_enc == 'laplacian':
            #     graph.ndata['lap_pos_enc'] = self.laplacian_positional_encoding(self.kg_ui_adj, self.conf['pos_enc_dim'])
        return graph


## Model

In [7]:
import dgl
import dgl.function as fn
import numpy as np
import networkx as nx 

from dgl.nn.pytorch.conv import HGTConv

class HGTransformer(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, num_heads, num_ntypes, num_etypes, n_layers, dropout, use_norm=True, residual=True, aggr='mean'):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hid_dim  = hid_dim
        self.n_layers = n_layers
        self.nheads = num_heads
        self.dropout = dropout 
        self.use_norm = use_norm
        self.residual = residual
        
        self.nt = num_ntypes
        self.et = num_etypes
        
        self.aggr_func =  aggr         
        if self.aggr_func == 'pool':
            self.conv = nn.Conv1d(in_channels=3, out_channels=1, kernel_size=3)  # Input channels: 3, Output channels: 1, Kernel size: 3
        self.layers = nn.ModuleList()
        if n_layers == 1:
            self.layers.append(HGTConv(in_dim, num_heads, out_dim, num_ntypes, num_etypes, dropout, use_norm))
        else:
            self.layers.append(HGTConv(in_dim, num_heads, hid_dim, num_ntypes, num_etypes, dropout, use_norm))
        for l in range(1, n_layers-1):
            self.layers.append(HGTConv(hid_dim*num_heads, num_heads, hid_dim, num_ntypes, num_etypes, dropout, use_norm))
        self.layers.append(HGTConv(hid_dim*num_heads, num_heads, out_dim, num_ntypes, num_etypes, dropout, use_norm))
    
    def forward(self, g, h):
        hidden_list = []
        
        for l in range(self.n_layers):
            h_ = self.layers[l](g, h, self.nt, self.et, presorted=True)
            # residual
            if self.residual:
                h+=  h_
            hidden_list.append(h)
        if self.aggr_func == 'sum':
            final_h = sum(hidden_list)
        elif self.aggr_func == 'mean':
            final_h = torch.mean(hidden_list)
        elif self.aggr_func == 'pool':
            final_h = self.conv(hidden_list.permute(0, 2, 1))
            final_h = final_h.squeeze(dim=1)
        return final_h 

In [8]:
# https://github.com/graphdeeplearning/graphtransformer
# add Laplacian Positional Encoding  

class LPHGTransformer(nn.Module):
    def __init__(self, config, data, args):
        super(LPHGTransformer, self).__init__()
        self.data = data
        adj = data.interaction_mat
        kg_adj = data.kg_interaction_mat
        self.adj  = TorchGraphInterface.convert_sparse_mat_to_tensor(ad
                                                                     j).to_dense().to(device)
        self.kg_adj = TorchGraphInterface.convert_sparse_mat_to_tensor(kg_adj).to_dense().to(device)
        
        self._parse_args(args)
        self.embedding_dict = self._init_model()
        
        self.g_ui = data.create_graph_with_pos_enc('user', 'item', self.embedding_dict['user_emb'], \
                                                   self.embedding_dict['item_emb'], type='interaction', pos_enc='laplacian')
        self.g_kg = data.create_graph_with_pos_enc('item', 'entity', self.embedding_dict['item_emb'], \
                                                   self.embedding_dict['entity_emb'], type='knowledge', pos_enc='laplacian')
        # ui lap_pos_enc
        self.lap_pos_enc_ui = data.lap_pos_enc_ui.to(device)
        sign_flip = torch.rand(self.lap_pos_enc_ui.size(1)).to(device)
        sign_flip[sign_flip>=0.5] = 1.0; sign_flip[sign_flip<0.5] = -1.0
        self.lap_pos_enc_ui = self.lap_pos_enc_ui * sign_flip.unsqueeze(0)
        
        # kg lap_pos_enc
        self.lap_pos_enc_kg = data.lap_pos_enc_kg.to(device)
        sign_flip = torch.rand(self.lap_pos_enc_kg.size(1)).to(device)
        sign_flip[sign_flip>=0.5] = 1.0; sign_flip[sign_flip<0.5] = -1.0
        self.lap_pos_enc_kg = self.lap_pos_enc_kg * sign_flip.unsqueeze(0)
        
        self.non_linear = nn.ReLU()
        self.dropout = nn.Dropout(self.drop_rate)
        
        if config['positional_enc'] == 'laplacian':
            pos_enc_dim = int(config['pos_enc_dim'])
            # encode ui 
            self.embedding_lap_pos_enc_ui = nn.Linear(pos_enc_dim, self.input_dim)
            # encode kg 
            self.embedding_lap_pos_enc_kg = nn.Linear(pos_enc_dim, self.input_dim) 
        
        self.lst_gnn_u = torch.nn.ModuleList()
        self.lst_gnn_i = torch.nn.ModuleList()
        self.lst_gnn_i2 = torch.nn.ModuleList()
        self.lst_gnn_e = torch.nn.ModuleList()
        
        self.ugformer_layers_ui = torch.nn.ModuleList()
        self.ugformer_layers_kg = torch.nn.ModuleList()
        
        self.layer_norms_ui = nn.ModuleList()
        self.layer_norms_kg = nn.ModuleList()
        self.layer_norms =  nn.ModuleList()
        
        # calculate n_nodes, n_edges
        
        num_ntypes_ui = data.num_ntypes_ui.to(device)
        num_etypes_ui = data.num_etypes_ui.to(device)
        
        num_ntypes_kg = data.num_ntypes_kg.to(device)
        num_etypes_kg = data.num_etypes_kg.to(device)
        
        for _layer in range(self.layers):
            # encoder_layers_ui = GraphTransformerEncoder(in_dim=self.input_dim, out_dim=self.hyper_dim, num_heads=self.nhead, dropout=self.drop_rate, \
            #                                           layer_norm=False, batch_norm=True, residual=True, use_bias=False, n_layers=self.num_self_att_layers)
            # encoder_layers_kg = GraphTransformerEncoder(in_dim=self.input_dim, out_dim=self.hyper_dim, num_heads=self.nhead, dropout=self.drop_rate, \
            #                                           layer_norm=False, batch_norm=True, residual=True, use_bias=False, n_layers=self.num_self_att_layers)
            encoder_layers_ui = HGTransformer(self.input_dim, self.input_dim, self.input_dim, self.nhead, num_ntypes_ui, num_etypes_ui, 1, self.drop_rate, use_norm=True, residual=True, aggr=self.aggr)
            encoder_layers_kg = HGTransformer(self.input_dim, self.input_dim, self.input_dim, self.nhead, num_ntypes_kg, num_etypes_kg, 1, self.drop_rate, use_norm=True, residual=True, aggr=self.aggr)
            
            self.ugformer_layers_ui.append(encoder_layers_ui)
            self.ugformer_layers_kg.append(encoder_layers_kg)
            
            self.layer_norms_ui.append(torch.nn.LayerNorm(self.input_dim))
            self.layer_norms_kg.append(torch.nn.LayerNorm(self.input_dim))
            self.layer_norms.append(torch.nn.LayerNorm(self.hyper_dim))
            
            self.lst_gnn_u.append( HGNNConv(leaky=self.p, input_dim=self.input_dim, hyper_dim=self.hyper_dim))
            self.lst_gnn_i.append( HGNNConv(leaky=self.p, input_dim=self.input_dim, hyper_dim=self.hyper_dim))
            self.lst_gnn_i2.append( HGNNConv(leaky=self.p, input_dim=self.input_dim, hyper_dim=self.hyper_dim))
            self.lst_gnn_e.append( HGNNConv(leaky=self.p, input_dim=self.input_dim, hyper_dim=self.hyper_dim))
        
    def _parse_args(self, args):
        self.input_dim = args['input_dim']
        self.hid_dim = args['hid_dim']
        self.hyper_dim = args['hyper_dim']
        self.p = args['p']
        self.drop_rate = args['drop_rate'] 
        self.layers = args['n_layers']
        self.nhead = args['n_heads']
        self.num_self_att_layers = args['n_self_att']
        self.aggr = args['aggr']
    
    def _init_model(self):
        initializer = nn.init.xavier_uniform_
        embedding_dict = nn.ParameterDict({
            'user_emb': nn.Parameter(initializer(torch.empty(self.data.user_num, self.input_dim)).to(device)),
            'item_emb': nn.Parameter(initializer(torch.empty(self.data.item_num, self.input_dim)).to(device)),
            'entity_emb': nn.Parameter(initializer(torch.empty(self.data.entity_num, self.input_dim)).to(device))
        })
        return embedding_dict
    
    def forward(self):
        uEmbed = self.embedding_dict['user_emb']
        iEmbed = self.embedding_dict['item_emb']
        eEmbed = self.embedding_dict['entity_emb']
        
        embeds = torch.cat([uEmbed, iEmbed, eEmbed], 0)
        embeds_ui = torch.cat([uEmbed, iEmbed], 0)
        embeds_kg = torch.cat([iEmbed, eEmbed], 0)

        all_embeddings = [embeds]
        
        h_lap_pos_enc_ui = self.embedding_lap_pos_enc_ui(self.lap_pos_enc_ui.float())  # num_nodes, input_dim
        h_lap_pos_enc_kg = self.embedding_lap_pos_enc_kg(self.lap_pos_enc_kg.float()) # num_nodes, input_dim
        import pdb; pdb.set_trace()
        
        embeds_ui += h_lap_pos_enc_ui # num_nodes, input_dim
        embeds_kg +=  h_lap_pos_enc_kg # num_nodes, input_dim
        
        for layer_idx in range(self.layers):
            uiEmbedTr = self.ugformer_layers_ui[layer_idx](self.g_ui, embeds_ui)
            kgEmbedTr = self.ugformer_layers_kg[layer_idx](self.g_kg, embeds_kg)
            
            if self.residual:
                embeds_ui += uiEmbedTr
                embeds_kg += kgEmbedTr
            if self.norm:
                embeds_ui = self.layer_norms_ui[i](embeds_ui)
                embeds_kg = self.layer_norms_kg[i](embeds_kg)
            
            hyperULat = self.lst_gnn_u[layer_idx](self.adj, uiEmbedTr[:self.data.user_num])
            hyperILat1 = self.lst_gnn_i[layer_idx](self.adj.T, uiEmbedTr[self.data.user_num:])
            hyperILat2 = self.lst_gnn_i2[layer_idx](self.kg_adj, kgEmbedTr[:self.data.item_num])
            hyperELat = self.lst_gnn_e[layer_idx](self.kg_adj.T, kgEmbedTr[self.data.item_num:])
            
            hyperILat = sum([hyperILat1, hyperILat2])
            
            ego_embeddings = torch.cat([hyperULat, hyperILat, hyperELat], dim=0)
            if self.norm:
                ego_embeddings = self.layer_norms[i](ego_embeddings)
                
            all_embeddings += [ego_embeddings]
        all_embeddings = torch.stack(all_embeddings, dim=1)
        all_embeddings = torch.mean(all_embeddings, dim=1)
        user_all_embeddings = all_embeddings[:self.data.user_num]
        item_all_embeddings = all_embeddings[self.data.user_num:self.data.user_num+ self.data.item_num]
        entity_all_embeddings = all_embeddings[self.data.user_num+ self.data.item_num:]
        return user_all_embeddings, item_all_embeddings, entity_all_embeddings


## Loss

In [9]:
def calculate_loss_ssl(anchor_emb, pos_emb, neg_emb, ent_emb, batch_size, reg, ssl_reg):
    calc_reg_loss = EmbLoss()
    rec_loss = bpr_loss(anchor_emb, pos_emb, neg_emb)
    reg_loss = reg * calc_reg_loss(anchor_emb, pos_emb, neg_emb, ent_emb) / batch_size 
    ssl_loss = ssl_reg * ''
    pass 

In [10]:
def calculate_loss(anchor_emb, pos_emb, neg_emb, ent_emb, batch_size, reg):
    calc_reg_loss = EmbLoss()
    rec_loss = bpr_loss(anchor_emb, pos_emb, neg_emb)
    reg_loss = reg * calc_reg_loss(anchor_emb, pos_emb, neg_emb, ent_emb) / batch_size
    return rec_loss, reg_loss 

In [11]:
def predict(u, rec, user_emb, item_emb):
    user_id  = rec.data.get_user_id(u)
    score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
    return score.cpu().numpy()  

## Train

In [12]:
def train():
    lst_train_losses = []
    lst_rec_losses = []
    lst_reg_losses = []
    lst_performances = []

    for ep in range(maxEpoch):
        train_losses = []
        rec_losses = []
        reg_losses = []

        
        for n, batch in enumerate(next_batch_pairwise_kg(rec.data, batchSize)):
            user_idx, pos_idx, neg_idx, e_idx = batch
            train_model.train()
            user_emb, item_emb, entity_emb = train_model()

            anchor_emb = user_emb[user_idx]
            pos_emb = item_emb[pos_idx]
            neg_emb = item_emb[neg_idx]
            ent_emb = entity_emb[e_idx]

            rec_loss, reg_loss = calculate_loss(anchor_emb, pos_emb, neg_emb, ent_emb, batchSize, reg)
            batch_loss = rec_loss + reg_loss 
            
            train_losses.append(batch_loss.item())
            rec_losses.append(rec_loss.item())
            reg_losses.append(reg_loss.item())
            optimizer.zero_grad()
            batch_loss.backward()
            torch.nn.utils.clip_grad_norm_(train_model.parameters(), 4)
            optimizer.step()

        batch_train_loss = np.mean(train_losses)
        scheduler.step(batch_train_loss)
        
        train_loss = np.mean(train_losses)
        rec_loss = np.mean(rec_losses)
        reg_loss = np.mean(reg_losses)

        lst_train_losses.append([ep, train_loss])
        lst_rec_losses.append([ep,rec_loss])
        lst_reg_losses.append([ep, reg_loss])

        # Evaluation
        train_model.eval()
        with torch.no_grad():
            user_emb, item_emb, _ = train_model()
#         rec.fast_evaluation(ep, user_emb, item_emb)
            _, data_ep = rec.fast_evaluation(train_model, ep, user_emb, item_emb)
        lst_performances.append(data_ep)

    rec.save_loss(lst_train_losses, lst_rec_losses, lst_reg_losses)
    rec.save_perfomance_training(lst_performances)
    user_emb, item_emb = rec.best_user_emb, rec.best_item_emb

    return user_emb, item_emb

## Test

In [78]:
def test(rec, user_emb, item_emb):
    def process_bar(num, total):
        rate = float(num) / total
        ratenum = int(50 * rate)
        r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
        sys.stdout.write(r)
        sys.stdout.flush()

    # predict
    rec_list = {}
    user_count = len(rec.data.test_set)
    for i, user in enumerate(rec.data.test_set):
        # s_find_candidates = time.time()
        candidates = predict(user, rec, user_emb, item_emb)
        # e_find_candidates = time.time()
        # print("Calculate candidates time: %f s" % (e_find_candidates - s_find_candidates))
        # predictedItems = denormalize(predictedItems, self.data.rScale[-1], self.data.rScale[0])
        rated_list, li = rec.data.user_rated(user)
        for item in rated_list:
            candidates[rec.data.item[item]] = -10e8

        # s_find_k_largest = time.time()
        ids, scores = find_k_largest(rec.max_N, candidates)
        # e_find_k_largest = time.time()
        # print("Find k largest candidates: %f s" % (e_find_k_largest - s_find_k_largest))
        item_names = [rec.data.id2item[iid] for iid in ids]
        rec_list[user] = list(zip(item_names, scores))
        if i % 1000 == 0:
            process_bar(i, user_count)
    process_bar(user_count, user_count)
    print('')
    rec.evaluate(rec_list)

## Main

In [107]:
model = 'HGNN'
if model not in ['HGNN', 'LightGCN']:
    print("No model found.")
config = ModelConf('./conf/' + model + '.conf')

dataset = 'ml-1m'
batchSize = int(config['batch_size'])
# lRate = float(config['learnRate'])
# maxEpoch = int(config['num.max.epoch'])
# reg = float(config['reg.lambda'])
maxEpoch = 500
lRates = [0.001, 0.001]
lrDecays = [0.9]
regs = [0.01, 0.001]
inputDims = [32]
hyperDims = [64, 128]
hidDims = [32, 64]
ps = [0.1,0.3,0.5]
dropRates = [0.1, 0.3, 0.5]
nLayers = [1, 2, 3]
nHeads = [1]
nSelfAtt = [1]
aggrs = ['sum', 'mean', 'pool']

if dataset == 'lastfm':
    training_set = "./dataset/lastfm/train.txt"
    test_set = './dataset/lastfm/test.txt'
    knowledge_data='./dataset/lastfm/processed/lastfm.kg'

elif dataset == 'ml-1m':
    training_set = './dataset/ml-1m/train.txt'
    test_set = './dataset/ml-1m/test.txt'
    knowledge_data ='./dataset/ml-1m/processed/ml-1m.kg'
hyperparameters = [lRates, lrDecays, regs, hyperDims, hidDims, inputDims, ps, dropRates, nLayers, nHeads, nSelfAtt, aggrs]

## Main Run

In [112]:
for params in product(*hyperparameters):
    lr, lr_decay, reg, hyper_dim, hid_dim, input_dim, prob, drop_rate, n_layers, n_heads, n_self_att, aggr = params
    print(params)
    args = {
        'lr': lr,
        'lr_decay': lr_decay,
        'reg': reg,
        'hid_dim': hid_dim,
        'hyper_dim': hyper_dim,
        'input_dim': input_dim,
        'p': prob,
        'drop_rate': drop_rate,
        'n_layers': n_layers,
        'input_dim': input_dim,
        'hyper_dim': hyper_dim,
        'n_heads': n_heads,
        'n_self_att': n_self_att,
        'aggr': aggr
    }
    training_data = FileIO.load_data_set(training_set, config['model.type'])
    test_data = FileIO.load_data_set(test_set, config['model.type'])
    knowledge_set = FileIO.load_kg_data(knowledge_data)

    rec = GraphRecommender(config, training_data, test_data, knowledge_set, **args)
    train_model = LPHGTransformer(rec.config, rec.data, args).to(device)
    optimizer  = torch.optim.Adam(train_model.parameters(), lr=lr)
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=lr_decay, patience=5)
    user_emb, item_emb, _ = train()
    test(rec, user_emb, item_emb)

    

(0.001, 0.9, 0.01, 64, 32, 32, 0.1, 0.1, 1, 1, 1, 'sum')
Loading the dataset ml-1m ....
constructing knowledge graph ...
> /tmp/ipykernel_20016/1465948094.py(8)__init__()
      6         Interaction.__init__(self, conf, training, test, knowledge_set, knowledge)
      7         import pdb; pdb.set_trace()
----> 8         self.lap_pos_enc_ui= self.laplacian_positional_encoding(self.ui_adj, self.conf['pos_enc_dim'])
      9         self.lap_pos_enc_kg = self.laplacian_positional_encoding(self.kg_ui_adj, self.conf['pos_enc_dim'])
     10 



ipdb>  self.ui_adj.shape


(9552, 9552)


ipdb>  self.kg_ui_adj


<78047x78047 sparse matrix of type '<class 'numpy.float32'>'
	with 366768 stored elements in Compressed Sparse Row format>


ipdb>  self.kg_ui_adj.shape


(78047, 78047)


ipdb>  q
